In [1]:
# !pip install -q selenium==4.3.0
# !pip install lxml
# !pip install -q beautifulsoup4==4.11.1
# !pip install -q backoff==2.1.2
# import backoff
# backoff.__version__
# !pip install -q pandas
# !pip install python-dotenv

In [2]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # 鍵盤事件
import pandas as pd
import backoff
from datetime import datetime
import pytz


tz = pytz.timezone('Asia/Taipei')
today = datetime.now(tz)
date_ = today.strftime("%Y%m%d_%H%M")

In [3]:
def set_ua():
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
    return user_agent

In [4]:
@backoff.on_exception(backoff.expo,
                        Exception, 
                      max_time=10)
def chrome_init():
    chrome_opt = webdriver.ChromeOptions()
    # chrome_opt.add_argument('--headless')
    chrome_opt.add_argument('--no-sandbox')
    chrome_opt.add_argument('--ignore-ssl-errors=yes')
    chrome_opt.add_argument('--ignore-certificate-errors')
    chrome_opt.add_argument(f'user-agent={set_ua()}')
    # chrome_opt.add_argument("--incognito")  # 使用無痕模式。用 selenium開瀏覽器已經很乾淨了，但疑心病重的可以用一下
    driver = webdriver.Remote(
        command_executor='http://selenium-hub:4444/wd/hub',
        options=chrome_opt
    )
    
    return driver

In [5]:
tags = ["action", "hack_and_slash", "action_run_jump", "fighting_martial_arts", "shmup", "action_fps", "action_tps", "arcade_rhythm", 
        "adventure", "casual", "story_rich", "visual_novel", "puzzle_matching", "hidden_object", "metroidvania",
        "rpg", "adventure_rpg", "rpg_action", "rpg_turn_based", "rpg_party_based", "rpg_jrpg", "rpg_strategy_tactics", "rogue_like_rogue_lite", 
        "simulation", "sim_hobby_sim", "sim_space_flight", "sim_building_automation", "sim_dating", "sim_physics_sandbox", "sim_life", "sim_farming_crafting", 
        "strategy", "strategy_card_board", "strategy_real_time", "strategy_turn_based", "strategy_cities_settlements", "tower_defense", "strategy_grand_4x", "strategy_military", 
        "sports_and_racing", "sports_individual", "sports_team", "sports", "racing", "racing_sim", "sports_sim", "sports_fishing_hunting", 
        "space", "horror", "mystery_detective", "anime", "survival", "science_fiction", "exploration_open_world", 
        "multiplayer_lan", "multiplayer_coop", "singleplayer", "multiplayer_local_party", "multiplayer", "multiplayer_mmo", "multiplayer_online_competitive"]

In [6]:
for x in tags:
    suc = False
    while suc is False:
        url = f'https://store.steampowered.com/category/{x}/?flavor=contenthub_topsellers' # 指定網址
        driver = chrome_init()
        driver.get(url) # 進入指定網址
        driver.implicitly_wait(5) # 等待伺服器反應最多 5 秒，如果在時間到之前反應就提早結束等待


        # pull pages
        n = 0
        while n < 70:
            try:
                butt = driver.find_element('css selector', '#SaleSection_13268 > div._1cOoCFwafBlSkwllIMf3XM._39HWXhhjsbML7K9sme9ItV.SaleSectionForCustomCSS > div._18byEIHFiivSklOwKqIx2b > div:nth-child(2) > div._3vjDu6zylspBzUE7FmM6Yl > div > div._36qA-3ePJIusV1oKLQep-w > button')
                if butt:
                    butt.click()
                    n += 1
                    print(n)
                    sleep(2)
                    
            except:
                break
        

        # 獲取頁面源代碼
        html_source = driver.page_source

        # 使用 BeautifulSoup 解析 HTML 內容
        soup = BeautifulSoup(html_source, 'lxml')

        all = soup.find_all("div", class_ = "v9uRg57bwOaPsvAnkXESO")

        columns = ['GameID', 'GameName', 'OperationDate', 'Rating_Summary', 'Total_Number_of_Rating', 'Cost', 'Tag', 'Category', 'Datetime']

        save_lst = []
        for rank in all:    

            appid_href = rank.find_all('div', class_='_3rrH9dPdtHVRMzAEw82AId')

            # get app id
            for appids in appid_href:
                appid = appids.find('a', href=True)
                if appid:
                    href = appid['href']
                    app_id = href.split('/')[4]

            d1 = today.strftime("%Y-%m-%d %H:%M")
            tags = rank.find_all('a', class_='_33yqka47vWurNqhnhLJb_m')
            name = rank.find('div', class_='_2ekpT6PjwtcFaT4jLQehUK StoreSaleWidgetTitle').text
            date = rank.find('div', class_='_1qvTFgmehUzbdYM9cw0eS7').text


            rating = rank.find('div', class_='_3ZWs0kB-1tuqQtie9KK-E7')
            if rating:
                comment = rating.find('div').text
                num_comment = rating.find('div', class_='_1wXL_MfRpdKQ3wZiNP5lrH').text
            else:
                comment = ""
                num_comment = ""
                
            costs = rank.find_all('div', class_='_3j4dI1yA7cRfCvK8h406OB')    
            for cost in costs:                                
                cost = cost.text

            value = []
            for tag in tags:                                
                value.append(tag.text)

                result = [app_id, name, date, comment, num_comment, cost, value, x, d1]
            
            save_lst.append(result)


        df = pd.DataFrame(
            data = save_lst,
            columns = columns
        )


        driver.quit()
        suc = True if len(df) != 0 else False

    df.to_csv(f'csv/steam_review_info_by_category/steam_review_info_{x}_{date_}.csv', index = 0, encoding = "utf-8")
    print(x, "Success Output")



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
action Success Output
1
2
3
4
5
6
7
8
9
10
11
12
13
14
hack_and_slash Success Output
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
action_run_jump Success Output
1
2
3
4
5
6
7
8
9
10
11
12
fighting_martial_arts Success Output
1
2
3
4
5
6
7
8
shmup Success Output
1
2
3
4
5
6
7
8
9
10
11
12
13
action_fps Success Output
1
2
3
4
5
action_tps Success Output
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
arcade_rhythm Success Output
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
adventure Success Output
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
3

In [7]:
# url = 'https://store.steampowered.com/category/arcade_rhythm/?flavor=contenthub_topsellers' # 指定網址
# driver = chrome_init()
# driver.get(url) # 進入指定網址
# driver.implicitly_wait(5) # 等待伺服器反應最多 5 秒，如果在時間到之前反應就提早結束等待

In [8]:
# n = 0

# while n < 25:
#     try:
#         butt = driver.find_element('css selector', '#SaleSection_13268 > div._1cOoCFwafBlSkwllIMf3XM._39HWXhhjsbML7K9sme9ItV.SaleSectionForCustomCSS > div._18byEIHFiivSklOwKqIx2b > div:nth-child(2) > div._3vjDu6zylspBzUE7FmM6Yl > div > div._36qA-3ePJIusV1oKLQep-w > button')
#         if butt:
#             butt.click()
#             n += 1
#             print(n)
#             sleep(2)
            
#     except:
#         break

In [9]:


# # 獲取頁面源代碼
# html_source = driver.page_source

# # 使用 BeautifulSoup 解析 HTML 內容
# soup = BeautifulSoup(html_source, 'lxml')

# all = soup.find_all("div", class_ = "v9uRg57bwOaPsvAnkXESO")

# columns = ['App_ID', 'Name', 'Operation_Date', 'Rating_Summary', 'Total_Num_Rating', 'Cost', 'Game_Tag', 'Datetime']
# today = datetime.today()

# save_lst = []
# for rank in all:    

#     appid_href = rank.find_all('div', class_='_3rrH9dPdtHVRMzAEw82AId')

#     # get app id
#     for appids in appid_href:
#         appid = appids.find('a', href=True)
#         if appid:
#             href = appid['href']
#             app_id = href.split('/')[4]

#     d1 = today.strftime("%Y-%m-%d %H:%M")
#     tags = rank.find_all('a', class_='_33yqka47vWurNqhnhLJb_m')
#     name = rank.find('div', class_='_2ekpT6PjwtcFaT4jLQehUK StoreSaleWidgetTitle').text
#     date = rank.find('div', class_='_1qvTFgmehUzbdYM9cw0eS7').text


#     rating = rank.find('div', class_='_3ZWs0kB-1tuqQtie9KK-E7')
#     if rating:
#         comment = rating.find('div').text
#         num_comment = rating.find('div', class_='_1wXL_MfRpdKQ3wZiNP5lrH').text
#     else:
#         comment = ""
#         num_comment = ""
        
#     costs = rank.find_all('div', class_='_3j4dI1yA7cRfCvK8h406OB')    
#     for cost in costs:                                
#         cost = cost.text

#     value = []
#     for tag in tags:                                
#         value.append(tag.text)

#         result = [app_id, name, date, comment, num_comment, cost, value, d1]
    
#     save_lst.append(result)


# df = pd.DataFrame(
#     data = save_lst,
#     columns = columns
# )


# driver.quit()

In [10]:

# df.to_csv('csv/steam_review_info_action.csv', index = 0, encoding = "utf-8")

In [11]:
# driver.quit()

In [12]:
# df

In [13]:
# tags = ["mystery_detective"]
# for x in tags:
#     url = f'https://store.steampowered.com/category/{x}/?flavor=contenthub_topsellers' # 指定網址
#     driver = chrome_init()
#     driver.get(url) # 進入指定網址
#     driver.implicitly_wait(5) # 等待伺服器反應最多 5 秒，如果在時間到之前反應就提早結束等待


#     # pull pages
#     n = 0
#     while n < 16:
#         try:
#             butt = driver.find_element('css selector', '#SaleSection_13268 > div._1cOoCFwafBlSkwllIMf3XM._39HWXhhjsbML7K9sme9ItV.SaleSectionForCustomCSS > div._18byEIHFiivSklOwKqIx2b > div:nth-child(2) > div._3vjDu6zylspBzUE7FmM6Yl > div > div._36qA-3ePJIusV1oKLQep-w > button')
#             if butt:
#                 butt.click()
#                 n += 1
#                 print(n)
#                 sleep(2)
                
#         except:
#             break
    

#     # 獲取頁面源代碼
#     html_source = driver.page_source

#     # 使用 BeautifulSoup 解析 HTML 內容
#     soup = BeautifulSoup(html_source, 'lxml')

#     all = soup.find_all("div", class_ = "v9uRg57bwOaPsvAnkXESO")

#     columns = ['App_Id', 'Name', 'Operation_Date', 'Rating_Summary', 'Total_Number_of_Rating', 'Cost', 'Game_Tag', 'Category', 'Datetime']

#     save_lst = []
#     for rank in all:    

#         appid_href = rank.find_all('div', class_='_3rrH9dPdtHVRMzAEw82AId')

#         # get app id
#         for appids in appid_href:
#             appid = appids.find('a', href=True)
#             if appid:
#                 href = appid['href']
#                 app_id = href.split('/')[4]

#         d1 = today.strftime("%Y-%m-%d %H:%M")
#         tags = rank.find_all('a', class_='_33yqka47vWurNqhnhLJb_m')
#         name = rank.find('div', class_='_2ekpT6PjwtcFaT4jLQehUK StoreSaleWidgetTitle').text
#         date = rank.find('div', class_='_1qvTFgmehUzbdYM9cw0eS7').text


#         rating = rank.find('div', class_='_3ZWs0kB-1tuqQtie9KK-E7')
#         if rating:
#             comment = rating.find('div').text
#             num_comment = rating.find('div', class_='_1wXL_MfRpdKQ3wZiNP5lrH').text
#         else:
#             comment = ""
#             num_comment = ""
            
#         costs = rank.find_all('div', class_='_3j4dI1yA7cRfCvK8h406OB')    
#         for cost in costs:                                
#             cost = cost.text

#         value = []
#         for tag in tags:                                
#             value.append(tag.text)

#             result = [app_id, name, date, comment, num_comment, cost, value, x, d1]
        
#         save_lst.append(result)


#     df = pd.DataFrame(
#         data = save_lst,
#         columns = columns
#     )


#     driver.quit()

#     df.to_csv(f'csv/steam_review_info_by_category/steam_review_info_{x}_{date_}.csv', index = 0, encoding = "utf-8")
#     print(x, "Success Output")